# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 21 2022 11:36AM,246937,15,VT80178076,"Virtus Pharmaceuticals, LLC",Released
1,Sep 21 2022 11:36AM,246937,15,VT80178077,"Virtus Pharmaceuticals, LLC",Released
2,Sep 21 2022 11:36AM,246937,15,VT80178078,"Virtus Pharmaceuticals, LLC",Released
3,Sep 21 2022 11:36AM,246937,15,VT80178131,"Virtus Pharmaceuticals, LLC",Released
4,Sep 21 2022 11:36AM,246937,15,VT80178132,"Virtus Pharmaceuticals, LLC",Released
5,Sep 21 2022 11:36AM,246937,15,VT80178133,"Virtus Pharmaceuticals, LLC",Released
6,Sep 21 2022 11:36AM,246937,15,VT80178134,"Virtus Pharmaceuticals, LLC",Released
7,Sep 21 2022 11:36AM,246937,15,VT80178135,"Virtus Pharmaceuticals, LLC",Released
8,Sep 21 2022 11:36AM,246937,15,VT80178136,"Virtus Pharmaceuticals, LLC",Released
9,Sep 21 2022 11:36AM,246937,15,VT80178137,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,246932,Released,5
37,246933,Released,1
38,246935,Released,20
39,246936,Executing,1
40,246937,Released,24


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246932,NaN,NaN,5.0
246933,NaN,NaN,1.0
246935,NaN,NaN,20.0
246936,NaN,1.0,NaN
246937,NaN,NaN,24.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246717,0.0,1.0,0.0
246828,42.0,32.0,3.0
246844,8.0,8.0,0.0
246845,7.0,0.0,2.0
246858,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246717,0,1,0
246828,42,32,3
246844,8,8,0
246845,7,0,2
246858,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246717,0,1,0
1,246828,42,32,3
2,246844,8,8,0
3,246845,7,0,2
4,246858,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246717,,1,
1,246828,42,32,3
2,246844,8,8,
3,246845,7,,2
4,246858,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 21 2022 11:36AM,246937,15,"Virtus Pharmaceuticals, LLC"
24,Sep 21 2022 11:36AM,246936,19,"GUSA Granules USA, Inc."
25,Sep 21 2022 11:34AM,246935,10,"Methapharm, Inc."
45,Sep 21 2022 11:34AM,246918,15,"Mizner Bioscience, LLC"
49,Sep 21 2022 11:31AM,246933,19,"Emersa Waterbox, LLC"
50,Sep 21 2022 11:28AM,246932,10,"Methapharm, Inc."
55,Sep 21 2022 11:00AM,246930,10,ISDIN Corporation
89,Sep 21 2022 10:55AM,246929,10,ISDIN Corporation
90,Sep 21 2022 10:48AM,246928,10,ISDIN Corporation
92,Sep 21 2022 10:39AM,246927,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 21 2022 11:36AM,246937,15,"Virtus Pharmaceuticals, LLC",,,24
1,Sep 21 2022 11:36AM,246936,19,"GUSA Granules USA, Inc.",,1,
2,Sep 21 2022 11:34AM,246935,10,"Methapharm, Inc.",,,20
3,Sep 21 2022 11:34AM,246918,15,"Mizner Bioscience, LLC",,,4
4,Sep 21 2022 11:31AM,246933,19,"Emersa Waterbox, LLC",,,1
5,Sep 21 2022 11:28AM,246932,10,"Methapharm, Inc.",,,5
6,Sep 21 2022 11:00AM,246930,10,ISDIN Corporation,,,34
7,Sep 21 2022 10:55AM,246929,10,ISDIN Corporation,,,1
8,Sep 21 2022 10:48AM,246928,10,ISDIN Corporation,,,2
9,Sep 21 2022 10:39AM,246927,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 11:36AM,246937,15,"Virtus Pharmaceuticals, LLC",24,,
1,Sep 21 2022 11:36AM,246936,19,"GUSA Granules USA, Inc.",,1,
2,Sep 21 2022 11:34AM,246935,10,"Methapharm, Inc.",20,,
3,Sep 21 2022 11:34AM,246918,15,"Mizner Bioscience, LLC",4,,
4,Sep 21 2022 11:31AM,246933,19,"Emersa Waterbox, LLC",1,,
5,Sep 21 2022 11:28AM,246932,10,"Methapharm, Inc.",5,,
6,Sep 21 2022 11:00AM,246930,10,ISDIN Corporation,34,,
7,Sep 21 2022 10:55AM,246929,10,ISDIN Corporation,1,,
8,Sep 21 2022 10:48AM,246928,10,ISDIN Corporation,2,,
9,Sep 21 2022 10:39AM,246927,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 11:36AM,246937,15,"Virtus Pharmaceuticals, LLC",24,,
1,Sep 21 2022 11:36AM,246936,19,"GUSA Granules USA, Inc.",,1,
2,Sep 21 2022 11:34AM,246935,10,"Methapharm, Inc.",20,,
3,Sep 21 2022 11:34AM,246918,15,"Mizner Bioscience, LLC",4,,
4,Sep 21 2022 11:31AM,246933,19,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 11:36AM,246937,15,"Virtus Pharmaceuticals, LLC",24.0,NaN,NaN
1,Sep 21 2022 11:36AM,246936,19,"GUSA Granules USA, Inc.",NaN,1.0,NaN
2,Sep 21 2022 11:34AM,246935,10,"Methapharm, Inc.",20.0,NaN,NaN
3,Sep 21 2022 11:34AM,246918,15,"Mizner Bioscience, LLC",4.0,NaN,NaN
4,Sep 21 2022 11:31AM,246933,19,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 11:36AM,246937,15,"Virtus Pharmaceuticals, LLC",24.0,0.0,0.0
1,Sep 21 2022 11:36AM,246936,19,"GUSA Granules USA, Inc.",0.0,1.0,0.0
2,Sep 21 2022 11:34AM,246935,10,"Methapharm, Inc.",20.0,0.0,0.0
3,Sep 21 2022 11:34AM,246918,15,"Mizner Bioscience, LLC",4.0,0.0,0.0
4,Sep 21 2022 11:31AM,246933,19,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4197449,119.0,11.0,14.0
12,246872,1.0,0.0,0.0
15,740771,59.0,0.0,0.0
16,740623,3.0,12.0,8.0
19,740792,8.0,1.0,0.0
20,740642,20.0,32.0,42.0
21,493595,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4197449,119.0,11.0,14.0
1,12,246872,1.0,0.0,0.0
2,15,740771,59.0,0.0,0.0
3,16,740623,3.0,12.0,8.0
4,19,740792,8.0,1.0,0.0
5,20,740642,20.0,32.0,42.0
6,21,493595,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,119.0,11.0,14.0
1,12,1.0,0.0,0.0
2,15,59.0,0.0,0.0
3,16,3.0,12.0,8.0
4,19,8.0,1.0,0.0
5,20,20.0,32.0,42.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,119.0
1,12,Released,1.0
2,15,Released,59.0
3,16,Released,3.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,14.0,0.0,0.0,8.0,0.0,42.0,0.0
Executing,11.0,0.0,0.0,12.0,1.0,32.0,1.0
Released,119.0,1.0,59.0,3.0,8.0,20.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,14.0,0.0,0.0,8.0,0.0,42.0,0.0
1,Executing,11.0,0.0,0.0,12.0,1.0,32.0,1.0
2,Released,119.0,1.0,59.0,3.0,8.0,20.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,14.0,0.0,0.0,8.0,0.0,42.0,0.0
1,Executing,11.0,0.0,0.0,12.0,1.0,32.0,1.0
2,Released,119.0,1.0,59.0,3.0,8.0,20.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()